In [2]:
import numpy as np
import cv2
import face_recognition as fr
import math

In [ ]:
prototxt_path = 'models/MobileNetSSD_deploy.prototxt'
model_path = 'models/MobileNetSSD_deploy.caffemodel'

min_confidence = 0.2

classes = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

person1 = fr.load_image_file("person1.jpg")
person2 = fr.load_image_file("Yogi.jpeg")
person3 = fr.load_image_file("Atharva.jpg")

person1_encodings = fr.face_encodings(shivang)[0]
person2_encoding = fr.face_encodings(yogi)[0]
person3_encoding = fr.face_encodings(atharva)[0]

known_faces = [shivang_encoding, yogi_encoding, atharva_encoding, sheshang_encoding, neel_encoding, sahaj_encoding, tanay_encoding, aarav_encoding, vedant_encoding]
names = ["person1", "person2", "person3"]

bottle_diagonal = [0, 0]
person_diagonal = [0, 0]
    
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    total_faces = 0
    distance = None
    
    height = int(cap.get(4))
    width = int(cap.get(3))
    
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007, (300, 300), 130)
    
    net.setInput(blob)
    detected_objects = net.forward()
    
    for i in range(detected_objects.shape[2]):
        confidence = detected_objects[0][0][i][2]
        
        if confidence > min_confidence:
            class_index = int(detected_objects[0, 0, i, 1])
            
            if class_index == 15:
#                 new_frame = frame[:, :, ::-1]
    
                locations = fr.face_locations(frame)
                face_encodings = fr.face_encodings(frame, locations)

                for (top, right, bottom, left), face_encoding in zip(locations, face_encodings):
                    matches = fr.compare_faces(known_faces, face_encoding)
                    total_faces = len(face_encodings)
                    name = "Unknown"

                    face_distances = fr.face_distance(known_faces, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index]:
                        name = names[best_match_index]
                    
                    person_diagonal[0] = (left + right) // 2
                    person_diagonal[1] = (top + bottom) // 2
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                    cv2.rectangle(frame, (left, bottom - 5), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    
                    # Case 01
                    if (bottle_diagonal[0] == 0 and bottle_diagonal[1] == 0):
                        cv2.putText(frame, str(name), (left + 4, bottom - 4), font, 1.0, (255, 255, 255), 1)
                    else: 
                        distance = math.sqrt(((person_diagonal[0] - bottle_diagonal[0]) ** 2) + ((person_diagonal[1] - bottle_diagonal[1]) ** 2))
                        if distance < 1000:
                            display = str(name) + " is holding a object"
                            cv2.putText(frame, display, (left + 4, bottom - 4), font, 1.0, (255, 255, 255), 1)
                            bottle_diagonal[0] = 0
                            bottle_diagonal[1] = 0
            
            else:
                upper_left_x = int(detected_objects[0, 0, i, 3] * width)
                upper_left_y = int(detected_objects[0, 0, i, 4] * width)
                lower_right_x = int(detected_objects[0, 0, i, 5] * width)
                lower_right_y = int(detected_objects[0, 0, i, 6] * width)
                
                
                bottle_diagonal[0] = (upper_left_x + upper_left_y) // 2
                bottle_diagonal[1] = (lower_right_x + lower_right_y) // 2
                 
    cv2.imshow("Detected objects", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()